In [1]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import extractFrames
import pathlib
import data
import cv2

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model, Sequential

In [2]:
def getPartialModel(savedModelPath):
    loadedModel = load_model(savedModelPath)

    indexOfGAPLayer = -1
    # Get the index of the last GlobalAveragePooling2D layer starting from the back of the list of layers
    for layer in loadedModel.layers[::-1]:
        layer_type = str(type(layer))
        if layer_type != "<class 'tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D'>":
            indexOfGAPLayer -= 1
        else:
            break

    if indexOfGAPLayer != -1:
        indexAfterGAPLayer = indexOfGAPLayer + 1
        partialModel = Sequential(loadedModel.layers[:indexAfterGAPLayer])
    else:
        # The GlobalAveragePooling2D Layer is the last layer. Thus, we get all the layers (no need to slice the list).
        partialModel = Sequential(loadedModel.layers[:])

    return partialModel

In [3]:
def saveFeatures(model, framesPath, videoFileName, destinationPath):
    def extractFeatures(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos") # shape: (299, 299, 3): # of dim: 3
        frame_arr = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        features  = model.predict(frame_arr)              # returns numpy array of shape: (1, 2048): # of dim: 2
        features  = features[0]                           # shape: (2048, ): # of dim: 1
        return features
    
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    numOfDigits = len(str(len(framesPaths)))
    padding     = f'%0{numOfDigits}d'
    for index, framePath in enumerate(framesPaths):
        sequencePath = pathlib.Path(destinationPath/(videoFileName + "_" + str(padding % index)))
        features = extractFeatures(model, framePath)
        features = np.expand_dims(features, axis = 0)
        print(f'Features from Frame # {index} saved at {sequencePath}')
        np.save(sequencePath, features)

In [4]:
def predictCNN(model, framesPath, classNames):
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    results = {}
    framesPaths = list(sorted(framesPath.glob("*.jpg")))
    for index, framePath in enumerate(framesPaths):
        predictions    = getPredictions(model, framePath)
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [5]:
def predictRNN(model, sequencesPath, classNames):
    def getPredictions(model, sequencePath):
        sequence      = np.load(sequencePath)
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    results = {}
    sequencesPaths = list(sorted(sequencesPath.glob("*.npy")))
    for index, sequencePath in enumerate(sequencesPaths):
        predictions    = getPredictions(model, sequencePath)
        topindex       = predictions.argsort()[-1:][0]
        predictedLabel = classNames[topindex]
        percentage     = predictions[topindex] * 100
        results[index] = [predictedLabel, percentage]
    return results

In [6]:
def createLabeledVideo(videoFilePath, videoFileName, outputPath, cnnPredictions, rnnPredictions):
    outputFilePath = str(outputPath/f'{videoFileName}_labeled.avi')
    vidCap = cv2.VideoCapture(videoFilePath) # Opens a video file for video capturing
    width  = int(vidCap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vidCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = int(vidCap.get(cv2.CAP_PROP_FOURCC)) # *'XVID'
#     fps    = int(vidCap.get(cv2.CAP_PROP_FPS))
    fps    = 8
    
    videoWriter = cv2.VideoWriter(outputFilePath, fourcc, fps, (width, height))
    tempDirectory = pathlib.Path(r"./temp")
    framesPath = tempDirectory/'Frames'
    frames = sorted(framesPath.glob("*"))
    for index, frame in enumerate(frames):
        frame = cv2.imread(str(frame))
        cv2.putText(frame, f'CNN: {cnnPredictions[index][0]} - {round(cnnPredictions[index][1], 2)}%', 
                    (1, 14), cv2.FONT_HERSHEY_DUPLEX , 0.45, (255, 100, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, f'RNN: {rnnPredictions[index][0]} - {round(rnnPredictions[index][1], 2)}%', 
                    (1, 30), cv2.FONT_HERSHEY_DUPLEX, 0.45, (255, 0, 100), 1, cv2.LINE_AA)
        videoWriter.write(frame)
    videoWriter.release()

In [7]:
# import random
# import cv2
# import pathlib

# videoFilePath   = r"D:\ActionRecognition\UCF-101\Archery\v_Archery_g01_c01.avi"
# videoFileName   = str(pathlib.Path(videoFilePath).stem)
# outputPath = tempDirectory = pathlib.Path(r"./")
# cnnPredictions = {}
# rnnPredictions = {}

# for i in range(200):
#     cnnPredictions[i] = ["qypZXCVBNM", random.uniform(0, 100)]
#     rnnPredictions[i] = ["Testing2", random.uniform(0, 100)]

# createLabeledVideo(videoFilePath, videoFileName, tempDirectory, cnnPredictions, rnnPredictions)

In [8]:
def main():
    dataObj    = data.Data()
    classNames = dataObj.classes
    
    tempDirectory = pathlib.Path(r"./temp")
    directoryFunctions.removeDirectory(tempDirectory)
    directoryFunctions.createDirectory(tempDirectory)
    
    videoFilePath   = r"D:\ActionRecognition\UCF-101\Archery\v_Archery_g01_c01.avi"
    videoFileName   = str(pathlib.Path(videoFilePath).stem)
    
    framesPath = tempDirectory/'Frames'
    directoryFunctions.createDirectory(framesPath)
    extractFrames.extractAllFrames(videoFilePath, videoFileName, str(framesPath))
    
    savedCNNModelPath = r"D:\ActionRecognition\Callbacks\CNN\3\ModelCheckpoint\1589059237_CNN_001_0.79.h5" # you can insert the path to a saved model here
    partialCNNModel   = getPartialModel(savedCNNModelPath)
    
    sequencesPath = tempDirectory/'Sequences'
    directoryFunctions.createDirectory(sequencesPath)
    saveFeatures(partialCNNModel, framesPath, videoFileName, sequencesPath)
    
    savedCNNModel  = load_model(savedCNNModelPath)
    cnnPredictions = predictCNN(savedCNNModel, framesPath, classNames)

    savedRNNModelPath = r"D:\ActionRecognition\Callbacks\RNN\3\ModelCheckpoint\1589230317_RNN_005_0.65.h5" # you can insert the path to a saved model here
    savedRNNModel     = load_model(savedRNNModelPath)
    rnnPredictions    = predictRNN(savedRNNModel, sequencesPath, classNames)
    
    createLabeledVideo(videoFilePath, videoFileName, tempDirectory, cnnPredictions, rnnPredictions)

In [9]:
main()

Creating... temp\Frames\v_Archery_g01_c01_000.jpg 0
Creating... temp\Frames\v_Archery_g01_c01_001.jpg 1
Creating... temp\Frames\v_Archery_g01_c01_002.jpg 2
Creating... temp\Frames\v_Archery_g01_c01_003.jpg 3
Creating... temp\Frames\v_Archery_g01_c01_004.jpg 4
Creating... temp\Frames\v_Archery_g01_c01_005.jpg 5
Creating... temp\Frames\v_Archery_g01_c01_006.jpg 6
Creating... temp\Frames\v_Archery_g01_c01_007.jpg 7
Creating... temp\Frames\v_Archery_g01_c01_008.jpg 8
Creating... temp\Frames\v_Archery_g01_c01_009.jpg 9
Creating... temp\Frames\v_Archery_g01_c01_010.jpg 10
Creating... temp\Frames\v_Archery_g01_c01_011.jpg 11
Creating... temp\Frames\v_Archery_g01_c01_012.jpg 12
Creating... temp\Frames\v_Archery_g01_c01_013.jpg 13
Creating... temp\Frames\v_Archery_g01_c01_014.jpg 14
Creating... temp\Frames\v_Archery_g01_c01_015.jpg 15
Creating... temp\Frames\v_Archery_g01_c01_016.jpg 16
Creating... temp\Frames\v_Archery_g01_c01_017.jpg 17
Creating... temp\Frames\v_Archery_g01_c01_018.jpg 18
Cre

Features from Frame # 18 saved at temp\Sequences\v_Archery_g01_c01_018
Features from Frame # 19 saved at temp\Sequences\v_Archery_g01_c01_019
Features from Frame # 20 saved at temp\Sequences\v_Archery_g01_c01_020
Features from Frame # 21 saved at temp\Sequences\v_Archery_g01_c01_021
Features from Frame # 22 saved at temp\Sequences\v_Archery_g01_c01_022
Features from Frame # 23 saved at temp\Sequences\v_Archery_g01_c01_023
Features from Frame # 24 saved at temp\Sequences\v_Archery_g01_c01_024
Features from Frame # 25 saved at temp\Sequences\v_Archery_g01_c01_025
Features from Frame # 26 saved at temp\Sequences\v_Archery_g01_c01_026
Features from Frame # 27 saved at temp\Sequences\v_Archery_g01_c01_027
Features from Frame # 28 saved at temp\Sequences\v_Archery_g01_c01_028
Features from Frame # 29 saved at temp\Sequences\v_Archery_g01_c01_029
Features from Frame # 30 saved at temp\Sequences\v_Archery_g01_c01_030
Features from Frame # 31 saved at temp\Sequences\v_Archery_g01_c01_031
Featur